# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=30

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==30]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm30', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm30/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-30 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]          [,3]        [,4]
sigmas -0.4831106659 1.0077427 -3.3135572216 22.14424550
       -0.4781156607 0.9971479 -3.2340826056 21.80148231
       -0.4731454816 0.9868316 -3.1561857646 21.46469481
       -0.4681998831 0.9767872 -3.0798276943 21.13372434
       -0.4632786232 0.9670081 -3.0049706183 20.80841599
       -0.4583814635 0.9574880 -2.9315779570 20.48861852
       -0.4535081692 0.9482209 -2.8596142961 20.17418434
       -0.4486585088 0.9392008 -2.7890453549 19.86496947
       -0.4438322541 0.9304220 -2.7198379558 19.56083348
       -0.4390291803 0.9218789 -2.6519599926 19.26163945
       -0.4342490658 0.9135662 -2.5853804003 18.96725391
       -0.4294916921 0.9054787 -2.5200691244 18.67754683
       -0.4247568439 0.8976111 -2.4559970911 18.39239147
       -0.4200443089 0.8899587 -2.3931361771 18.11166444
       -0.4153538778 0.8825166 -2.3314591805 17.83524553

...

[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.648992574 1.2601878 -1.819471553  2.769362195
       -0.636596213 1.2486396 -1.804336258  2.785459814
       -0.624351642 1.2371735 -1.788992392  2.800984529
       -0.612255189 1.2257913 -1.773445481  2.815937542
       -0.600303313 1.2144951 -1.757701046  2.830320143
       -0.588492598 1.2032866 -1.741764608  2.844133709
       -0.576819750 1.1921679 -1.725641683  2.857379705
       -0.565281587 1.1811405 -1.709337782  2.870059687
       -0.553875036 1.1702063 -1.692858409  2.882175294
       -0.542597128 1.1593670 -1.676209061  2.893728258
       -0.531444994 1.1486242 -1.659395225  2.904720395
       -0.520415860 1.1379795 -1.642422377  2.915153610
       -0.509507042 1.1274344 -1.625295981  2.925029894
       -0.498715943 1.1169906 -1.608021487  2.934351326
       -0.488040050 1.1066494 -1.590604331  2.943120070
       -0.477476928 1.0964122 -1.573

        0.379263798 0.7083412  0.329276101  0.706691463
        0.383714780 0.7105562  0.333416365  0.673024243
        0.388146039 0.7127964  0.337317119  0.639197171
        0.392557749 0.7150600  0.340977092  0.605211065
        0.396950081 0.7173455  0.344395013  0.571066724
        0.401323204 0.7196511  0.347569618  0.536764928
        0.405677287 0.7219751  0.350499642  0.502306440
        0.410012494 0.7243159  0.353183826  0.467692008
        0.414328988 0.7266718  0.355620914  0.432922362
        0.418626929 0.7290411  0.357809653  0.397998219
        0.422906478 0.7314220  0.359748793  0.362920281
        0.427167790 0.7338128  0.361437087  0.327689238
        0.431411020 0.7362118  0.362873291  0.292305767
        0.435636321 0.7386172  0.364056166  0.256770535
        0.439843844 0.7410272  0.364984475  0.221084198
        0.444033738 0.7434401  0.365656983  0.185247400
        0.448206151 0.7458542  0.366072461  0.149260781
        0.452361226 0.7482675  0.366229683  0.11

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_25817/236840523.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]          [,3]        [,4]
sigmas -0.927392416 1.3549513 -2.2982620566  3.63500224
       -0.907356639 1.3373563 -2.2752923311  3.65308291
       -0.887714421 1.3199033 -2.2519741106  3.67027058
       -0.868450600 1.3025964 -2.2283177674  3.68656616
       -0.849550871 1.2854399 -2.2043337062  3.70197079
       -0.831001726 1.2684378 -2.1800323595  3.71648585
       -0.812790395 1.2515940 -2.1554241826  3.73011291
       -0.794904793 1.2349123 -2.1305196486  3.74285378
       -0.777333473 1.2183966 -2.1053292437  3.75471048
       -0.760065581 1.2020503 -2.0798634626  3.76568523
       -0.743090814 1.1858771 -2.0541328035  3.77578046
       -0.726399387 1.1698804 -2.0281477643  3.78499880
       -0.709981996 1.1540634 -2.0019188371  3.79334309
       -0.693829788 1.1384293 -1.9754565047  3.80081634
       -0.677934332 1.1229813 -1.9487712356  3.80742176
       -0.662287594 1.1077222 -1.9218734801  3.81316276
       -0.6468

        0.437231087 0.7155874  0.4078115349 -0.50830905
        0.442388067 0.7188101  0.4027043169 -0.56336243
        0.447518588 0.7219857  0.3971376254 -0.61850506
        0.452622922 0.7251104  0.3911114249 -0.67373210
        0.457701334 0.7281805  0.3846257436 -0.72903872
        0.462754085 0.7311924  0.3776806732 -0.78442012
        0.467781435 0.7341423  0.3702763686 -0.83987151
        0.472783637 0.7370266  0.3624130480 -0.89538812
        0.477760941 0.7398416  0.3540909919 -0.95096520
        0.482713594 0.7425837  0.3453105436 -1.00659802
        0.487641839 0.7452491  0.3360721085 -1.06228187
        0.492545916 0.7478343  0.3263761536 -1.11801205
        0.497426060 0.7503356  0.3162232076 -1.17378389
        0.502282504 0.7527494  0.3056138602 -1.22959275
        0.507115477 0.7550722  0.2945487616 -1.28543400
        0.511925205 0.7573004  0.2830286224 -1.34130303
        0.516711909 0.7594303  0.2710542130 -1.39719527
        0.521475811 0.7614585  0.2586263627 -1.4

In [2]:
print(error_i)

[2, 3, 5, 7, 11, 13, 25, 48]
